# Blockchain Explained In Simple Python Code 

### by Vincenzo Furcillo

So, what is a blockchain? In simple words, a blockchain is an append-only database, divided in blocks, in which each block contains a set of transactions. These blocks are usually created in regular intervals, and every block is linked to the previous block in the chain. 

In Python, a block can be written as class in this way: (Shift + Enter to execute)

In [24]:
class Block:
    height = None
    transactions = None
    parent_height = None
    parent_hash = None

The height of the first block of the blockchain, also called the genesis block, is 0. The genesis block constitutes the foundation of the blockchain, and each block is then stacked on top of the other resulting in a layered structure. Using this visualization, the "height" refers to the distance from the first block. 

Besides the transactions, the block also must contain a reference to the previous block, called parent, so that the blocks can be linked together to guarantee untamperability.

The primary identifier of a block is its cryptographic hash. This can be computed using an hashing algorithm on the block's information. The output of this hashing algorithm is a unique 32-byte hash string.

In order to apply this algorithm, we import the hashlib library, which contains the hash algorithms, and the json library to deserialise the hash to a string: (Shift + Enter to execute)

In [25]:
import hashlib
import json

Now we have everything we need to start experimenting with transactions and blocks. Let's start by creating our genesis block, and set the first transaction:

In [26]:
block_Genesis = Block()
block_Genesis.height = 0
block_Genesis.transactions = []

This being the first block of our blockchain, it has no parent block. Now we can initiate a transaction:

In [27]:
transaction_A = 'Alice sends 5 euros to Bob'

The pool of pending transaction now looks like the following:

In [28]:
transactions = [transaction_A]
print(transactions)

['Alice sends 5 euros to Bob']


Blockchain validators (or "miners") are listening to this pool of transactions, and compete between each other to group them together into a valid block. Once this happens, the block is formed and propagated to everyone in the network:

In [29]:
block_A = Block()
block_A.height = 1
block_A.transactions = [transaction_A]

The block_A is our first "real" block, and it contains the transaction we previously initialised:

In [30]:
print(block_A.transactions)

['Alice sends 5 euros to Bob']


The only thing we are missing now is the link to the previous block. As previously mentioned, we need to use an hashing algorithm on the previous block information. We can do it in this way:

In [31]:
block_A.parent_hash = hashlib.sha256(json.dumps(block_Genesis.__dict__).encode('utf-8')).hexdigest()
print(block_A.parent_hash)

7f42ab3758f50c368b32488380d6b31910c880ff6b2b8fa9cdb64ec48921a486


Once you execute the code above, you should see this hash string appear: "7f42ab3758f50c368b32488380d6b31910c880ff6b2b8fa9cdb64ec48921a486". This hash string represents the hash of the genesis block, and it is added to the next block, block_A, in the parent_hash field. If any information in the previous block changes even slightly, the hash string computed will turn out completely different, making it evident that the blockchain has been tampered with. 

Let's show this blockchain property with an example. First we add more transactions and more blocks to our blockchain:

In [32]:
transaction_B = 'Bob sends 5 euros to Charlie'
transaction_C = 'Alice sends 10 euros to Dennis'

In [33]:
block_B = Block()
block_B.height = 2
block_B.transactions = [transaction_B,transaction_C]
block_B.parent_id = block_A.height
block_B.parent_hash = hashlib.sha256(json.dumps(block_A.__dict__).encode('utf-8')).hexdigest()

In [34]:
transaction_D = 'Charlie sends 5 euros to Alice'

In [35]:
block_C = Block()
block_C.height = 3
block_C.transactions = [transaction_D]
block_C.parent_id = block_B.height
block_C.parent_hash = hashlib.sha256(json.dumps(block_B.__dict__).encode('utf-8')).hexdigest()

Executing the code above we now have 3 blocks (4 with the genesis block) all linked to each other through their parent_hash property. Let's print the parent hash in block C:

In [36]:
print("Parent hash in block C:")
print(block_C.parent_hash)

Parent hash in block C:
962b77a80e18848086d95247aa197b3f7cfe5155eee4ea853f3a2c6ca6f7f1f8


Now let's try and modify the data in one the previous blocks:

In [37]:
print("original data in block A: ")
print(block_A.transactions)
print("Change data to: ")
block_A.transactions = 'Alice sends 5 euros to Vincenzo'
print(block_A.transactions)

original data in block A: 
['Alice sends 5 euros to Bob']
Change data to: 
Alice sends 5 euros to Vincenzo


And recalculate the parent hash for the blocks:

In [38]:
block_B.parent_hash = hashlib.sha256(json.dumps(block_A.__dict__).encode('utf-8')).hexdigest()
block_C.parent_hash = hashlib.sha256(json.dumps(block_B.__dict__).encode('utf-8')).hexdigest()

print("New parent hash in block C:")
print(block_C.parent_hash)

New parent hash in block C:
4cf13f5f3c5ac80060280c53510e58fdf50f511a2336d6599db20dc64ef479c9


The new hash is: "4cf13f5f3c5ac80060280c53510e58fdf50f511a2336d6599db20dc64ef479c9"
which is totally different from the orginal hash from above:
"962b77a80e18848086d95247aa197b3f7cfe5155eee4ea853f3a2c6ca6f7f1f8", clearly showing that the blockchain has been tampered with. But how can the tampering be prevented in the first place?

## Securing the blockchain

The blockchain is secured by validators (or "miners"), which get rewarded tokens for appending valid blocks to the blockchain. Nodes in the network are also responsible for checking every transaction, and propagating only the legitimate ones to other nodes. If a miner tries to include an invalid transaction in a block, this attempt will be detected and their entire block will be ignored by everyone. In this case, the miners will not get their reward. 

Moreover, in a proof of work (PoW) blockchain the miners are required to do some puzzle work before they can add blocks to the blockchain. Aggregating the transactions and prepare a new block is the easy part of a miner's job. However, once the block is ready, a miner needs to compute the hash of the block as we have done in the previous section. In real scenario however, the output of the hashing function is required to fall within certain boundaries, which determine the difficulty of the mining process. In particular, the hash of the block data, plus a random number, called nonce, should have a certain number of leading zeros in order to be considered valid. Therefore, the goal for miners is to find the nonce that, hashed together with the block data, returns an output that satisfies the difficulty condition.

To show this process visually, let's initialise a block's data:

In [39]:
import hashlib

blockData_string = '{"transaction": "Alice sends 5 to Bob", "parent_id": 3, "id": 4}'
blockData_bytes = bytes(blockData_string, 'utf-8')


Now that we have the block data, we need to find the right nonce. To do this, we iterate nonce after nonce, each time checking if the hashed output satisfies the difficulty condition, which we will set to 3 leading zero ("000"). For example we can take the block's data,

In [40]:
print(blockData_bytes)

b'{"transaction": "Alice sends 5 to Bob", "parent_id": 3, "id": 4}'


 Add a random number (nonce), hash the pair of values and observe the hash:

In [41]:
nonce = 15 # change this number to try different values
nonce = str(nonce).encode('utf-8') #encoding
print(nonce)
result = hashlib.sha256(blockData_bytes + nonce).hexdigest()
print(result)

b'15'
a11a3203f79fd3a52aa01a74bb2b99780da1254aad5f2f859f16947d83c53028


As the hash does not have 3 leading zeros, it does not satissfy the difficulty condition and cannot be accepted. You can try yourself giving different values to the nonce above, and see how the hash varies totally randomly, even despite only a slight change in the input.

And finally, we can simply iterate with a for loop over different nonce values until we find the valid hash with the predetermined number of leading zeros:

In [42]:
for i in range(1000000):
    nonce = str(i).encode('utf-8')
    result = hashlib.sha256(blockData_bytes + nonce).hexdigest()
    print(result)
    if result[0:3] == '000':
        print("The nonce : " + str(i))
        print("Output hash which has 3 leading zeros: " + result)
        break

...
The nonce : 551
Output hash which has 3 leading zeros: 0002a9dd37305cb122e87c4443b4344a85cdf778ae950be52dfc5678d928022b


The hash and the nonce are both added to the block information, so that every node can now verify the block by running the hash function using the block data and the provided nonce as inputs.

At the time of writing, the bitcoin difficulty target is set to 19 leading zeros for the block hash, making it extremely hard to compute. You can play with the code above yourself, increasing the number of zeros required and see of the number of iterations increase accordingly.

This means that if someone wants to alter the history of the blockchain by replacing some blocks, these extremely computationally expensive puzzles need to be solved, and they need to be solved in a very short time. In fact, by the time a block is modified, the nonce found and the hash computed, other miners would have likely added more blocks to the blockchain. Since miners add blocks to the longest blockchain, the tampered block would consequently be ignored, and the tampering effort would become worthless anyway.

The incentives are therefore pushing miners to behave honestly. Miners would try and assemble a new candidate block, work hard to find the answer (nonce) in the proof of work, and then add it to the longest chain as quickly as possible. This ensures that the miners secure their reward, aligning the actions of miners with the security of the network, while simultaneously implementing the monetary supply through the issuance of new tokens.